# Stable Diffusion Interactive Notebook 📓 🤖

A widgets-based interactive notebook for Google Colab that lets users generate AI images from prompts (Text2Image) using [Stable Diffusion (by Stability AI, Runway & CompVis)](https://en.wikipedia.org/wiki/Stable_Diffusion).

This notebook aims to be an alternative to WebUIs while offering a simple and lightweight GUI for anyone to get started with Stable Diffusion.

Uses Stable Diffusion, [HuggingFace](https://huggingface.co/) Diffusers and [Jupyter widgets](https://github.com/jupyter-widgets/ipywidgets).



In [8]:
#@title 👇 Installing dependencies { display-mode: "form" }
#@markdown ---
#@markdown Make sure to select **GPU** as the runtime type:<br/>
#@markdown *Runtime->Change Runtime Type->Under Hardware accelerator, select GPU*
#@markdown
#@markdown ---
#torch diffusers transformers xformers
#!pip -q install  install xformers
!pip install xformers


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
     -- ------------------------------------- 0.5/7.8 MB 5.6 MB/s eta 0:00:02
     --------------------- ------------------ 4.2/7.8 MB 14.8 MB/s eta 0:00:01
     ------------------------------------- -- 7.3/7.8 MB 22.7 MB/s eta 0:00:01
     ---------------------------------------  7.6/7.8 MB 13.4 MB/s eta 0:00:01
     ---------------------------------------- 7.8/7.8 MB 10.4 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\Alex\\AppData\\Local\\Temp\\pip-install-m47q_0p_\\xformers_99110a6d028c4b5ebed86650a82c68f8\\third_party\\flash-attention\\csrc\\composable_kernel\\client_example\\24_grouped_conv_activation\\grouped_convnd_bwd_data_bilinear\\grouped_conv_bwd_data_bilinear_residual_fp16.cpp'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths



In [ ]:
#@title 👇 Selecting Model { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Select Model** - A list of Stable Diffusion models to choose from.
#@markdown - **Select Sampler** - A list of schedulers to choose from. Default is EulerAncestralScheduler.
#@markdown - **Safety Checker** - Enable/Disable uncensored content
#@markdown
#@markdown ---

from diffusers import StableDiffusionPipeline, EulerAncestralDiscreteScheduler, DDIMScheduler, EulerDiscreteScheduler, UniPCMultistepScheduler
from diffusers.models import AutoencoderKL
import torch
import ipywidgets as widgets
import importlib

#Enable third party widget support
# from google.colab import output
# output.enable_custom_widget_manager()

#Pipe
pipe = None

#Models
select_model = widgets.Dropdown(
    options=[
        ("Stable Diffusion 2.1 Base" , "stabilityai/stable-diffusion-2-1-base"),
        ("Stable Diffusion 2.1" , "stabilityai/stable-diffusion-2-1"),
        ("Stable Diffusion 1.5", "runwayml/stable-diffusion-v1-5"),
        ("Dreamlike Photoreal 2.0" , "dreamlike-art/dreamlike-photoreal-2.0"),
        ("OpenJourney v4" , "prompthero/openjourney-v4")
    ],
    description="Select Model:"
)

#Schedulers
select_sampler = widgets.Dropdown(
    options=[
        "EulerAncestralDiscreteScheduler",
        "EulerDiscreteScheduler",
        "UniPCMultistepScheduler",
        "DDIMScheduler"
    ],
    description="Select Schedular:"
)
select_sampler.style.description_width = "auto"

#Safety Checker
safety_check = widgets.Checkbox(
    value=True,
    description="Enable Safety Check",
    layout=widgets.Layout(margin="0px 0px 0px -85px")
)

#Output
out = widgets.Output()

#Apply Settings
apply_btn = widgets.Button(
    description="Apply",
    button_style="info"
)


#Get scheduler
# Get scheduler
def get_scheduler(name):
    if name == "EulerAncestralDiscreteScheduler":
        return EulerAncestralDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")
    elif name == "DDIMScheduler":
        return DDIMScheduler.from_pretrained(select_model.value, subfolder="scheduler")
    elif name == "EulerDiscreteScheduler":
        return EulerDiscreteScheduler.from_pretrained(select_model.value, subfolder="scheduler")
    elif name == "UniPCMultistepScheduler":
        return UniPCMultistepScheduler.from_pretrained(select_model.value, subfolder="scheduler")
    else:
        raise ValueError(f"Unknown scheduler name: {name}")

#Run pipeline
def pipeline(p):

  global pipe

  out.clear_output()
  apply_btn.disabled = True

  with out:

    print("Running, please wait...")

    pipe = StableDiffusionPipeline.from_pretrained(
      select_model.value,
      scheduler=get_scheduler(select_sampler.value),
      torch_dtype=torch.float16,
      vae=AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse", torch_dtype=torch.float16).to("cuda")
    ).to("cuda")

    if not safety_check.value:
      pipe.safety_checker = None

    pipe.enable_xformers_memory_efficient_attention()

    print("Finished!")

  apply_btn.disabled = False


#Display
apply_btn.on_click(pipeline)

widgets.VBox(
    [
      widgets.HTML(value="<h2>Configure Pipeline</h2>"),
      select_model, select_sampler, safety_check, apply_btn, out
    ]
)


A matching Triton is not available, some optimizations will not be enabled
Traceback (most recent call last):
  File "c:\Users\Alex\anaconda3\envs\GAN\lib\site-packages\xformers\__init__.py", line 57, in _is_triton_available
    import triton  # noqa
ModuleNotFoundError: No module named 'triton'


In [6]:
#@title 👇 Generating Images { form-width: "20%", display-mode: "form" }
#@markdown ---
#@markdown - **Prompt** - Description of the image
#@markdown - **Negative Prompt** - Things you don't want to see or ignore in the image
#@markdown - **Steps** - Number of denoising steps. Higher steps may lead to better results but takes longer time to generate the image. Default is `30`.
#@markdown - **CFG** - Guidance scale ranging from `0` to `20`. Lower values allow the AI to be more creative and less strict at following the prompt. Default is `7.5`.
#@markdown - **Seed** - A random value that controls image generation. The same seed and prompt produce the same images. Set `-1` for using random seed values.
#@markdown ---
import ipywidgets as widgets, mediapy, random
import IPython.display


#PARAMETER WIDGETS
width = "300px"

prompt = widgets.Textarea(
    value="",
    placeholder="Enter prompt",
    #description="Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

neg_prompt = widgets.Textarea(
    value="",
    placeholder="Enter negative prompt",
    #description="Negative Prompt:",
    rows=5,
    layout=widgets.Layout(width="600px")
)

num_images = widgets.IntText(
    value=1,
    description="Images:",
    layout=widgets.Layout(width=width),
)

steps = widgets.IntText(
    value=30,
    description="Steps:",
    layout=widgets.Layout(width=width)
)

CFG = widgets.FloatText(
    value=7.5,
    description="CFG:",
    layout=widgets.Layout(width=width)
)

img_height = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Height:",
    layout=widgets.Layout(width=width)
)

img_width = widgets.Dropdown(
    options=[('512px', 512), ('768px', 768)],
    value=512,
    description="Width:",
    layout=widgets.Layout(width=width)
)

random_seed = widgets.IntText(
    value=-1,
    description="Seed:",
    layout=widgets.Layout(width=width),
    disabled=False
)

generate = widgets.Button(
    description="Generate",
    disabled=False,
    button_style="primary"
)

display_imgs = widgets.Output()


#RUN
def generate_img(i):

  #Clear output
  display_imgs.clear_output()
  generate.disabled = True

  #Calculate seed
  seed = random.randint(0, 2147483647) if random_seed.value == -1 else random_seed.value

  with display_imgs:

    print("Running...")

    images = pipe(
      prompt.value,
      height = img_height.value,
      width = img_width.value,
      num_inference_steps = steps.value,
      guidance_scale = CFG.value,
      num_images_per_prompt = num_images.value,
      negative_prompt = neg_prompt.value,
      generator = torch.Generator("cuda").manual_seed(seed),
    ).images
    mediapy.show_images(images)

    print(f"Seed:\n{seed}")

  generate.disabled = False

#Display
generate.on_click(generate_img)

widgets.VBox(
    [
      widgets.AppLayout(
        header=widgets.HTML(
            value="<h2>Stable Diffusion</h2>",
        ),
        left_sidebar=widgets.VBox(
            [num_images, steps, CFG, img_height, img_width, random_seed]
        ),
        center=widgets.VBox(
            [prompt, neg_prompt, generate]
        ),
        right_sidebar=None,
        footer=None
      ),
      display_imgs
    ]
)